In [1]:
from competition_utils import *
from models.base_config import BaseConfig

# 제출 함수를 리스트로 정의 (submission_N_학번)
# 깃허브에는 최대 3개 업로드 데모시 하이퍼파라미터 튜닝 후 1개만 제출
# submission_functions = ['submission_YOLOv8n', 'submission_YOLOv9t', 'submission_YOLOv10n', 'submission_YOLO11n', 'submission_YOLO12n',
#                         'submission_YOLOXNano', 'submission_YOGAn', 'submission_LightweightOB', 'submission_FLDetn', 'submission_HyperYOLOt'] 
submission_functions = ['submission_YOLOv8n', 'submission_HyperYOLOt'] 
for submission_function in submission_functions:
    exec(f"from submissions.{submission_function} import {submission_function}")

# 분석 방향에 따라 iteration 수 수정
# 평가시에는 데모와 다른 스플릿 시드를 만들어 [1, 10]로 진행 예정
iterations = [1, 2] 
# 'Crown Detection'는 예제 데이터셋, 데모 및 평가시에는 'CV_Competition'
Dataset_Name = 'airplane' 

# 결과를 모아 하나의 csv 파일에 저장
results_df = pd.DataFrame(columns=[
    'Experiment Time', 'Iteration', 'Submission Function', 
    'IoU', 'Dice', 'Precision', 'Recall', 'Output Json Path',
])
csv_filename = os.path.join(BaseConfig.get_output_dir(), f"Evaluation_Results_{BaseConfig.get_experiment_time()}.csv")
json_foldername = "json"
vis_foldername = "vis"

os.makedirs(os.path.join(BaseConfig.get_output_dir(), json_foldername), exist_ok=True)
os.makedirs(os.path.join(BaseConfig.get_output_dir(), vis_foldername), exist_ok=True)

# 서로 다른 iteration, submission function으로 실험 진행
for iteration in range(iterations[0], iterations[1]+1):
    
    yaml_path = f'Datasets/{Dataset_Name}/data_iter_{iteration:02d}.yaml'
    
    for submission_function in submission_functions:
        
        ex_time = datetime.now().strftime('%y%m%d_%H%M%S')
        output_json_path = os.path.join(BaseConfig.get_output_dir(), json_foldername, f"{ex_time}_{submission_function}_Iter_{iteration}_detection_results.json")
        
        globals()[submission_function](yaml_path, output_json_path)
        
        labels_dir = f"Datasets/{Dataset_Name}/labels"  
        vis_output_dir = os.path.join(BaseConfig.get_output_dir(), vis_foldername, f"{ex_time}_visualization_results")
        image_level_result_path =os.path.join(BaseConfig.get_output_dir(), json_foldername, f"{ex_time}_{submission_function}_Iter_{iteration}_image_level_results.json")
        stats = eval_and_vis(yaml_path, output_json_path, labels_dir, image_level_result_path, vis_output_dir, vis=True) # 분석 방향에 따라 vis=True 설정
        new_row = {
            'Experiment Time': ex_time,
            'Iteration': iteration,
            'Submission Function': submission_function,
            'IoU': stats['IoU']['avg'],
            'Dice': stats['Dice']['avg'],
            'Precision': stats['Precision']['avg'],
            'Recall': stats['Recall']['avg'],
            'Output Json Path': output_json_path,
        }
        results_df = pd.concat([results_df, pd.DataFrame([new_row])], ignore_index=True)
        results_df.to_csv(csv_filename, index=False)

Ultralytics 8.3.96 🚀 Python-3.10.16 torch-2.8.0.dev20250417+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=Datasets/airplane/data_iter_01.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=output/250422_005007/train, name=250422_005008_yolov8n_airplane_Iter_1, exist_ok=False, pretrained=False, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=True, close_mosaic=2, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fals

train: Scanning Datasets/airplane/labels... 1051 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1051/1051 [00:00<00:00, 1175.96it/s]

train: New cache created: Datasets/airplane/labels.cache



val: Scanning Datasets/airplane/labels... 351 images, 0 backgrounds, 0 corrupt: 100%|██████████| 351/351 [00:00<00:00, 735.68it/s]

val: New cache created: Datasets/airplane/labels.cache


...

      20/20      4.87G      1.019     0.9658      2.013         11        640: 100%|██████████| 66/66 [00:06<00:00,  9.93it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:01<00:00,  5.86it/s]

                   all        351        386      0.698      0.619      0.674      0.333
                   all        351        386      0.698      0.619      0.674      0.333



20 epochs completed in 0.051 hours.

20 epochs completed in 0.051 hours.
Optimizer stripped from output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/weights/last.pt, 6.3MB
Optimizer stripped from output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/weights/last.pt, 6.3MB
Optimizer stripped from output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/weights/best.pt, 6.3MB
Optimizer stripped from output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/weights/best.pt, 6.3MB

Validating output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/weights/best.pt...

Validating output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/weights/best.pt...
Hyper-YOLOv1.0 🚀 Python-3.10.16 torch-2.8.0.dev20250417+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
Hyper-YOLOv1.0 🚀 Python-3.10.16 torch-2.8.0.dev20250417+cu128 CUDA:0 (NVIDIA GeForce RTX 5070 Ti, 16303MiB)
Hyper-YOLOt summary (fused): 214 layers, 3004179 par

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  4.18it/s]


                   all        351        386      0.699      0.617      0.674      0.333
                   all        351        386      0.699      0.617      0.674      0.333
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Speed: 0.2ms preprocess, 0.8ms inference, 0.0ms loss, 1.0ms postprocess per image
Saving output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/predictions.json...
Saving output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/predictions.json...

Evaluating pycocotools mAP using output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/predictions.json and /home/gamejoongsa/CV_midterm_competition_code/Datasets/airplane/annotations/instances_val2017.json...

Evaluating pycocotools mAP using output/250422_005007/train/250422_010313_HyperYOLOt_airplane_Iter_2/predictions.json and /home/gamejoongsa/CV_midterm_competition_code/Datasets/airplane/annotations/instances_val2017.json...
pycocotools unable

In [ ]:
# 분석 방향에 따라 전체 결과를 모아 평균-표준편차-통계테스트 결과 저장 
keep_columns = ['Iteration','Submission Function']
keep_measures = ['IoU','Dice','Precision','Recall']  

reduction = 'Iteration'
row = 'Measure Type'
column = 'Submission Function'
reference_column = 'submission_YOLOv8n'

custom_fmt_template = '{mean_fmt} ± {std_fmt} {significance}'
significance_levels = [0.1, 0.05, 0.01]
decimal_places = 3
transpose = True
make_tables(csv_filename, keep_columns, keep_measures, reduction, row, column,  
            reference_column, custom_fmt_template, significance_levels, decimal_places, transpose)